In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
# import matplotlib.pyplot as plt

In [93]:
df = pd.read_csv('log.csv')
df.head()

,cycle,mode,ball_x,ball_y,ball_vx,ball_vy,ball_area,my_x,my_y,my_vx,...,opp_player_4_kick,opp_player_4_dist,opp_player_4_area,opp_player_5_x,opp_player_5_y,opp_player_5_vx,opp_player_5_vy,opp_player_5_kick,opp_player_5_dist,opp_player_5_area
0,421,play_on,-26.4266,-28.4263,-1.8165,0.4374,1,-40.6874,-5.3881,-0.0766,...,0,29.220594,0,-20.7132,-29.4389,-0.2120,0.0566,0,31.263551,0
1,422,play_on,-28.2004,-27.9994,-1.6675,0.4013,1,-40.9578,-6.1488,-0.1082,...,0,29.738884,0,-21.4979,-29.2106,-0.2911,0.0847,0,30.175061,0
2,423,play_on,-29.8278,-27.5923,-1.5297,0.3826,1,-41.2283,-7.0106,-0.1082,...,0,28.885839,0,-22.9478,17.4070,-0.0603,0.0012,0,30.502391,0
3,424,play_on,-31.3757,-27.2134,-1.4551,0.3562,1,-41.5153,-7.9394,-0.1148,...,0,28.603467,0,-23.4223,16.9508,-0.1954,-0.1878,0,30.771394,0
4,425,play_on,-32.8175,-26.8181,-1.3553,0.3716,1,-41.8329,-8.7905,-0.1270,...,0,28.318141,0,-23.9871,16.3070,-0.2325,-0.2650,0,30.795407,0


one-hot encode the mode column

In [94]:
df = pd.get_dummies(df, columns=['mode']).astype('float32')
df = df.astype('float32')
df = df.drop('cycle', axis=1)
df.head()

,ball_x,ball_y,ball_vx,ball_vy,ball_area,my_x,my_y,my_vx,my_vy,my_dash_power,...,mode_foul_charge_r,mode_free_kick_l,mode_free_kick_r,mode_goal_kick_l,mode_goal_r,mode_indirect_free_kick_r,mode_kick_in_l,mode_kick_in_r,mode_offside_r,mode_play_on
0,-26.426600,-28.426300,-1.8165,0.4374,1.0,-40.687401,-5.3881,-0.0766,-0.2327,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-28.200399,-27.999399,-1.6675,0.4013,1.0,-40.957802,-6.1488,-0.1082,-0.3043,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-29.827801,-27.592300,-1.5297,0.3826,1.0,-41.228298,-7.0106,-0.1082,-0.3447,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-31.375700,-27.213400,-1.4551,0.3562,1.0,-41.515301,-7.9394,-0.1148,-0.3715,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-32.817501,-26.818100,-1.3553,0.3716,1.0,-41.832901,-8.7905,-0.1270,-0.3404,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


normalize the data

In [95]:
for col in df.columns:
    if col != 'my_dash_power' and col != 'my_dash_dir' and col != 'my_turn':
        df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

split the data into train and test

In [96]:
train = df.sample(frac=0.8, random_state=200)

split the data into train and validation

In [97]:
validation = df.drop(train.index)
train.head()

,ball_x,ball_y,ball_vx,ball_vy,ball_area,my_x,my_y,my_vx,my_vy,my_dash_power,...,mode_foul_charge_r,mode_free_kick_l,mode_free_kick_r,mode_goal_kick_l,mode_goal_r,mode_indirect_free_kick_r,mode_kick_in_l,mode_kick_in_r,mode_offside_r,mode_play_on
71589,0.971587,0.699369,0.375538,0.487628,0.333333,0.429493,0.632545,0.345225,0.460793,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
34897,0.145410,0.846977,0.484202,0.407590,0.666667,0.126713,0.643489,0.502596,0.496011,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
71431,0.229355,0.998804,0.494190,0.501477,0.666667,0.143500,0.645084,0.502596,0.496011,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
41764,0.875269,0.806967,0.700593,0.245082,0.333333,0.261055,0.609565,0.570558,0.461249,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7596,0.998483,0.001081,0.494190,0.501477,0.333333,0.333459,0.416931,0.355724,0.492706,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


add layers to the model

In [98]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(len(df.columns) - 3,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),  # drop 20% of the neurons
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),  # drop 20% of the neurons
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='linear')
])

In [99]:
model.compile(optimizer='adam', loss='mse')

In [100]:
model.fit(train.drop(columns=['my_dash_power', 'my_dash_dir', 'my_turn'], axis=1), train.loc[:,['my_dash_power', 'my_dash_dir', 'my_turn']], epochs=100, validation_data=(validation.drop(['my_dash_power', 'my_dash_dir', 'my_turn'], axis=1), validation.loc[:,['my_dash_power', 'my_dash_dir', 'my_turn']]))

Epoch 1/100
2002/2002 [==============================] - 3s 1ms/step - loss: 1592.8442 - val_loss: 1516.0833
Epoch 2/100
2002/2002 [==============================] - 3s 1ms/step - loss: 1528.9968 - val_loss: 1469.1289
Epoch 3/100
2002/2002 [==============================] - 3s 1ms/step - loss: 1503.0374 - val_loss: 1470.6202
Epoch 4/100
2002/2002 [==============================] - 3s 1ms/step - loss: 1489.7552 - val_loss: 1455.1908
Epoch 5/100
2002/2002 [==============================] - 3s 1ms/step - loss: 1483.6923 - val_loss: 1475.4335
Epoch 6/100
2002/2002 [==============================] - 3s 1ms/step - loss: 1476.3219 - val_loss: 1443.6859
Epoch 7/100
2002/2002 [==============================] - 3s 1ms/step - loss: 1469.3901 - val_loss: 1444.5623
Epoch 8/100
2002/2002 [==============================] - 3s 1ms/step - loss: 1467.6996 - val_loss: 1433.5996
Epoch 9/100
2002/2002 [==============================] - 3s 1ms/step - loss: 1462.9568 - val_loss: 1433.6307
Epoch 10/100
2002/2

In [79]:
model.save('model.h5')

/Users/hasan/Documents/final-project/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [80]:
model.evaluate(validation.drop(['my_dash_power', 'my_dash_dir', 'my_turn'], axis=1), validation.loc[:,['my_dash_power', 'my_dash_dir', 'my_turn']])

501/501 [==============================] - 0s 356us/step - loss: 1409.6647 - mse: 1409.6647


[1409.6646728515625, 1409.6646728515625]

In [81]:
model.predict(validation.drop(['my_dash_power', 'my_dash_dir', 'my_turn'], axis=1))

501/501 [==============================] - 0s 323us/step


array([[ 89.13286   ,   0.30985585, -20.11398   ],
       [ 91.31894   ,   2.3412523 , -15.146709  ],
       [ 79.151344  ,  -0.5145242 ,  -9.072315  ],
       ...,
       [ 26.573235  ,  -0.24591227,  -6.1114573 ],
       [ 64.194664  ,   2.9087675 ,  29.494766  ],
       [ 22.539923  ,   3.4484525 ,  42.01024   ]], dtype=float32)

In [82]:
result_df = validation.loc[:,['my_dash_power', 'my_dash_dir', 'my_turn']]

In [83]:
result_df.head(3)

,my_dash_power,my_dash_dir,my_turn
3,100.0,6.0,37.0
4,100.0,4.0,-39.0
14,100.0,0.0,-49.0


save in result.csv

In [84]:
result_df.to_csv('result.csv', index=False)